<h3> Loading train_data for training models


In [82]:
# loading data from txt file in JSON format

import json

train_file = open('train.txt')
train_data = json.load(train_file)

In [83]:
# creating alias for more direct call

train_t = train_data['t']
train_xs = train_data['c_{XS}']
train_x = train_data['c_{X}']
train_x_2 = train_data['c_{X_2}']

n_exp = len(train_xs)
print(n_exp)
n_t = len(train_xs[0])
print(n_t)
print(train_t)


50
101
[0.0, 0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6000000000000001, 0.7000000000000001, 0.8, 0.9, 1.0, 1.1, 1.2000000000000002, 1.3, 1.4000000000000001, 1.5, 1.6, 1.7000000000000002, 1.8, 1.9000000000000001, 2.0, 2.1, 2.2, 2.3000000000000003, 2.4000000000000004, 2.5, 2.6, 2.7, 2.8000000000000003, 2.9000000000000004, 3.0, 3.1, 3.2, 3.3000000000000003, 3.4000000000000004, 3.5, 3.6, 3.7, 3.8000000000000003, 3.9000000000000004, 4.0, 4.1000000000000005, 4.2, 4.3, 4.4, 4.5, 4.6000000000000005, 4.7, 4.800000000000001, 4.9, 5.0, 5.1000000000000005, 5.2, 5.300000000000001, 5.4, 5.5, 5.6000000000000005, 5.7, 5.800000000000001, 5.9, 6.0, 6.1000000000000005, 6.2, 6.300000000000001, 6.4, 6.5, 6.6000000000000005, 6.7, 6.800000000000001, 6.9, 7.0, 7.1000000000000005, 7.2, 7.300000000000001, 7.4, 7.5, 7.6000000000000005, 7.7, 7.800000000000001, 7.9, 8.0, 8.1, 8.200000000000001, 8.3, 8.4, 8.5, 8.6, 8.700000000000001, 8.8, 8.9, 9.0, 9.1, 9.200000000000001, 9.3, 9.4, 9.5, 9.600000000000001, 9.70000

In [84]:
# plotting C_xs for data identification 

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np


for i in range(n_exp):
    break 
    fig, axs = plt.subplots(1,3,figsize=(9,3))
    axs[0].set_ylim(0,1)
    axs[1].set_ylim(0,1)
    axs[2].set_ylim(0,1)
    axs[0].plot(train_t,train_x[i])
    axs[0].set_title('Quantity of X')
    axs[1].plot(train_t,train_xs[i])
    axs[1].set_title('Quantity of XS')
    axs[2].plot(train_t,train_x_2[i])
    axs[2].set_title('Quantity of X_2')
    plt.show()



<h3> Building models



In [85]:
from torch import nn

class FirstModel(nn.Module):
    def __init__(self) -> None:
        super(FirstModel,self).__init__()
        self.linear = nn.Linear(3,3)
        self.relu = nn.ReLU()
    
    def forward(self,x):
        x = self.linear(x)
        x = self.relu(x)
        x = self.linear(x)
        x = self.relu(x)
        return x 


<h3> Creating dataloaders


In [86]:
import torch

In [87]:
data_in , data_out = [] , []

for i in range(n_exp):
    for j in range(n_t-1):
        data_in.append([train_x[i][j],train_xs[i][j],train_x_2[i][j]])
        data_out.append([train_x[i][j+1],train_xs[i][j+1],train_x_2[i][j+1]])

from sklearn.model_selection import train_test_split



data_in = torch.FloatTensor(data_in)
data_out = torch.FloatTensor(data_out)

train_in , val_in , train_out , val_out = train_test_split(data_in,data_out,test_size=0.3,random_state=42)

print(len(train_in))
print(len(val_in))


3500
1500


In [88]:
class Data(torch.utils.data.Dataset):
    def __init__(self,inp,out) -> None:
        self.x = inp 
        self.y = out

    def __len__(self):
        return len(self.x)

    def __getitem__(self,idx):
        return self.x[idx] , self.y[idx]

In [89]:
train , val = Data(train_in,train_out) , Data(val_in,val_out)

train_dataloader = torch.utils.data.DataLoader(train,batch_size=32)
val_dataloader = torch.utils.data.DataLoader(val, batch_size=32)

<h3> Train models

In [90]:


model = FirstModel()

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

model = model.to(device)
criterion = criterion.to(device)

In [92]:
tolerance = 1e-10 

from tqdm import tqdm
min_val_loss=100000000000
for epoch in range(10):
  total_loss_train=0
  total_acc_train=0
  for x, y in tqdm(train_dataloader):
    x = x.to(device)
    y = y.to(device)

    output = model(x.float())

    batch_loss = criterion(output, y)
    total_loss_train += batch_loss.item()
    
    acc = ((y-output).absolute()<tolerance).sum().item()

    total_acc_train += acc

    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()
  
  total_loss_val=0
  total_acc_val=0

  with torch.no_grad():
    for x, y in tqdm(val_dataloader):

      x = x.to(device)
      y = y.to(device)

      output = model(x.float())

      batch_loss = criterion(output, y)
      total_loss_val += batch_loss.item()

      
      acc = ((y-output).absolute()<tolerance).sum().item()
      total_acc_val += acc
  
  print(
      f'Epochs: {epoch+1} | Train Loss: {total_loss_train / len(train):.3f}\
      | Train Accuracy: {total_acc_train/len(train):.3f}\
      | Val Loss: {total_loss_val/len(val):.3f}\
      | Val Accuracy:{total_acc_val/len(val):.3f}'
  )

  if min_val_loss>total_loss_val/len(val):
    min_val_loss = total_loss_val/len(val)
    torch.save(model.state_dict(), "simplemodel.pt")
    print(f"Save model because val loss improve loss {min_val_loss:.3f}")


100%|██████████| 47/47 [00:00<00:00, 772.99it/s]


Epochs: 1 | Train Loss: 0.032      | Train Accuracy: 0.000      | Val Loss: 0.031      | Val Accuracy:0.000
Save model because val loss improve loss 0.031


100%|██████████| 47/47 [00:00<00:00, 1550.98it/s]


Epochs: 2 | Train Loss: 0.031      | Train Accuracy: 0.000      | Val Loss: 0.031      | Val Accuracy:0.000
Save model because val loss improve loss 0.031


100%|██████████| 47/47 [00:00<00:00, 1626.95it/s]


...

Epochs: 8 | Train Loss: 0.030      | Train Accuracy: 0.000      | Val Loss: 0.030      | Val Accuracy:0.000
Save model because val loss improve loss 0.030


100%|██████████| 47/47 [00:00<00:00, 1684.17it/s]


Epochs: 9 | Train Loss: 0.030      | Train Accuracy: 0.000      | Val Loss: 0.030      | Val Accuracy:0.000
Save model because val loss improve loss 0.030


100%|██████████| 47/47 [00:00<00:00, 1387.31it/s]

Epochs: 10 | Train Loss: 0.030      | Train Accuracy: 0.000      | Val Loss: 0.030      | Val Accuracy:0.000
Save model because val loss improve loss 0.030
